### Importing games data from basketball reference

In [4]:
import pandas as pd
import numpy as np
from faker import Faker
import os

fake = Faker()
np.random.seed(20)

# File paths for saving/loading data
file_paths = {
    "games": "games.csv",
    "customers": "customers.csv",
    "food_items": "food_items.csv",
    "merchandise_items": "merchandise_items.csv",
    "ticket_sales": "ticket_sales.csv",
    "food_beverage_sales": "food_beverage_sales.csv",
    "merchandise_sales": "merchandise_sales.csv",
    "seating_chart": "seating_chart.csv"
}


# Helper function to save a DataFrame to CSV
def save_to_csv(df, filename):
    df.to_csv(filename, index=False)

# Helper function to check if file exists and load it
def load_or_generate(file_path, generator_function):
    if os.path.exists(file_path):
        try:
            print(f"Loading existing data from {file_path}...")
            return pd.read_csv(file_path)
        except Exception as e:
            print(f"Error loading {file_path}: {e}. Generating new data...")
            return generator_function()
    else:
        print(f"No existing file found for {file_path}. Generating data...")
        return generator_function()

# Function to generate games table
def generate_games():
    games = pd.read_excel('2023-2024-season.xlsx')  # Replace with your actual file path
    games['H/A'] = games['H/A'].replace({'@': 'Away'}).fillna('Home')
    games.fillna({'Reg/OT': "Reg", 'Notes': "Regular Season"}, inplace=True)

    # Add Max Capacity and Attendance Percentage
    max_capacity = 18581
    games['MaxCapacity'] = max_capacity
    games['AttendancePercentage'] = (games['Attendance'] / max_capacity * 100).round(2)

    # Add GameID for uniqueness
    games['GameID'] = range(1, len(games) + 1)

    save_to_csv(games, file_paths['games'])
    return games

# Function to generate customers table
def generate_customers():
    customers = pd.DataFrame({
        'CustomerID': range(1, 20001),
        'Name': [fake.name() for _ in range(20000)],
        'Age': np.random.randint(18, 65, 20000),
        'TotalSpend ($)': np.random.randint(500, 10000, 20000)
    })
    customers['Email'] = customers['Name'].str.replace(' ', '.').str.lower() + "@example.com"
    save_to_csv(customers, file_paths['customers'])
    return customers

# Function to generate seating chart data (simplified for levels)
def generate_seating_chart():
    seating_levels = ["Courtside", "Lower Bowl", "Plaza Level", "Upper Bowl", "Super Boxes"]
    seating_chart = pd.DataFrame({
        "SeatingType": seating_levels
    })
    save_to_csv(seating_chart, file_paths['seating_chart'])
    return seating_chart

def generate_ticket_sales():
    games = pd.read_csv(file_paths['games'])
    customers = pd.read_csv(file_paths['customers'])

    # Define ticket price ranges for each seating type
    ticket_price_ranges = {
        "Courtside": (1500, 1800),
        "Lower Bowl": (300, 700),
        "Plaza Level": (150, 300),
        "Upper Bowl": (50, 120),
        "Super Boxes": (200, 400)
    }

    ticket_sales = []

    for game_id, attendance in zip(games['GameID'], games['Attendance']):
        allocation = {
            "Courtside": int(attendance * 0.05),
            "Lower Bowl": int(attendance * 0.30),
            "Plaza Level": int(attendance * 0.40),
            "Upper Bowl": int(attendance * 0.20),
            "Super Boxes": int(attendance * 0.05),
        }

        # Adjust allocation to match attendance
        total_allocated = sum(allocation.values())
        if total_allocated < attendance:
            allocation["Upper Bowl"] += attendance - total_allocated

        for seating_type, num_tickets in allocation.items():
            for _ in range(num_tickets):
                customer_id = np.random.choice(customers["CustomerID"])
                ticket_price = np.random.randint(
                    ticket_price_ranges[seating_type][0],
                    ticket_price_ranges[seating_type][1] + 1
                )
                ticket_sales.append({
                    'TicketID': len(ticket_sales) + 1,
                    'GameID': game_id,
                    'CustomerID': customer_id,
                    'SeatingType': seating_type,
                    'TicketPrice ($)': ticket_price
                })

    ticket_sales_table = pd.DataFrame(ticket_sales)
    save_to_csv(ticket_sales_table, file_paths["ticket_sales"])
    return ticket_sales_table


# Function to generate food items table
def generate_food_items():
    food_items = pd.DataFrame({
        'FoodItemID': range(1, 16),
        'FoodName': ['Hot Dog', 'Nachos', 'Burger', 'Fries', 'Pizza Slice',
                     'Popcorn', 'Pretzel', 'Chicken Tenders', 'Soda', 'Beer',
                     'Wine', 'Water Bottle', 'Candy', 'Ice Cream', 'Coffee'],
        'Price ($)': [5, 7, 10, 4, 8, 6, 5, 12, 3, 8, 10, 2, 3, 6, 4]
    })
    save_to_csv(food_items, file_paths['food_items'])
    return food_items

# Function to generate merchandise items table
def generate_merchandise_items():
    merchandise_items = pd.DataFrame({
        'ProductID': range(1, 19),
        'ProductName': ['Home Jersey', 'Away Jersey', 'Baseball Cap', 'Scarf', 'Keychain',
                        'Mug', 'Sticker Pack', 'Autographed Ball', 'Team Poster', 'Backpack',
                        'Hoodie', 'T-Shirt', 'Team Jacket', 'Bobblehead', 'Phone Case',
                        'Socks', 'Team Flag', 'Water Bottle'],
        'Price ($)': [80, 85, 25, 20, 10, 15, 5, 150, 12, 50, 60, 25, 100, 30, 18, 10, 20, 12]
    })
    save_to_csv(merchandise_items, file_paths['merchandise_items'])
    return merchandise_items

def generate_food_beverage_sales():
    games = pd.read_csv(file_paths['games'])
    customers = pd.read_csv(file_paths['customers'])
    food_items = pd.read_csv(file_paths['food_items'])
    
    food_beverage_sales = []
    for game_id, attendance in zip(games['GameID'], games['Attendance']):
        # Number of transactions
        num_transactions = np.random.randint(int(attendance * 0.2), int(attendance * 0.5))
        
        for _ in range(num_transactions):
            customer_id = np.random.choice(customers['CustomerID'])
            num_items = np.random.randint(1, 4)  # Number of items in the transaction
            selected_items = np.random.choice(food_items['FoodItemID'], num_items, replace=False)
            
            for item_id in selected_items:
                item_price = food_items.loc[food_items['FoodItemID'] == item_id, 'Price ($)'].values[0]
                food_beverage_sales.append({
                    'OrderID': len(food_beverage_sales) + 1,
                    'GameID': game_id,
                    'CustomerID': customer_id,
                    'FoodItemID': item_id,
                    'AmountSpent ($)': item_price
                })
    
    food_beverage_sales_table = pd.DataFrame(food_beverage_sales)
    save_to_csv(food_beverage_sales_table, file_paths['food_beverage_sales'])
    return food_beverage_sales_table


# Function to generate merchandise sales
def generate_merchandise_sales():
    games = pd.read_csv(file_paths['games'])
    customers = pd.read_csv(file_paths['customers'])
    merchandise_items = pd.read_csv(file_paths['merchandise_items'])
    
    merchandise_sales = []
    
    for game_id, attendance in zip(games['GameID'], games['Attendance']):
        # Number of transactions
        num_transactions = np.random.randint(int(attendance * 0.1), int(attendance * 0.4))
        
        for _ in range(num_transactions):
            customer_id = np.random.choice(customers['CustomerID'])
            num_items = np.random.randint(1, 4)  # Number of items in the transaction
            selected_items = np.random.choice(merchandise_items['ProductID'], num_items, replace=False)
            
            for product_id in selected_items:
                product_price = merchandise_items.loc[merchandise_items['ProductID'] == product_id, 'Price ($)'].values[0]
                merchandise_sales.append({
                    'TransactionID': len(merchandise_sales) + 1,
                    'GameID': game_id,
                    'CustomerID': customer_id,
                    'ProductID': product_id,
                    'AmountSpent ($)': product_price
                })
    
    merchandise_sales_table = pd.DataFrame(merchandise_sales)
    save_to_csv(merchandise_sales_table, file_paths['merchandise_sales'])
    return merchandise_sales_table


# Main function to generate or load all tables
def main():
    games = load_or_generate(file_paths['games'], generate_games)
    customers = load_or_generate(file_paths['customers'], generate_customers)
    seating_chart = load_or_generate(file_paths['seating_chart'], generate_seating_chart)
    ticket_sales = load_or_generate(file_paths['ticket_sales'], generate_ticket_sales)
    food_items = load_or_generate(file_paths['food_items'], generate_food_items)
    food_beverage_sales = load_or_generate(file_paths['food_beverage_sales'], generate_food_beverage_sales)
    merchandise_items = load_or_generate(file_paths['merchandise_items'], generate_merchandise_items)
    merchandise_sales = load_or_generate(file_paths['merchandise_sales'], generate_merchandise_sales)

    # Print confirmation that all tables are ready
    print("All tables have been generated or loaded successfully.")

    # Optionally, display the first few rows of each dataset for verification
    print("\nSample Data:")
    print("Games Table:\n", games.head())
    print("Customers Table:\n", customers.head())
    print("Seating Chart Table:\n", seating_chart.head())
    print("Ticket Sales Table:\n", ticket_sales.head())
    print("Food Items Table:\n", food_items.head())
    print("Food & Beverage Sales Table:\n", food_beverage_sales.head())
    print("Merchandise Items Table:\n", merchandise_items.head())
    print("Merchandise Sales Table:\n", merchandise_sales.head())

if __name__ == "__main__":
    main()


Loading existing data from games.csv...
Loading existing data from customers.csv...
Loading existing data from seating_chart.csv...
No existing file found for ticket_sales.csv. Generating data...
Loading existing data from food_items.csv...
Loading existing data from food_beverage_sales.csv...
Loading existing data from merchandise_items.csv...
Loading existing data from merchandise_sales.csv...
All tables have been generated or loaded successfully.

Sample Data:
Games Table:
    GameID             Date Start (ET)   H/A              Opponent W/L Reg/OT  \
0       1  Wed Oct 25 2023      9:30p  Home      Dallas Mavericks   L    Reg   
1       2  Fri Oct 27 2023      8:00p  Home       Houston Rockets   W     OT   
2       3  Sun Oct 29 2023      9:00p  Away  Los Angeles Clippers   L    Reg   
3       4  Tue Oct 31 2023     10:00p  Away          Phoenix Suns   W    Reg   
4       5   Thu Nov 2 2023     10:00p  Away          Phoenix Suns   W    Reg   

   Spurs-Score  Opp-Score Record  Att